# *WoMG*: tutorial

This is a tutorial for *WoMG*. The WoMG software generates synthetic datasets of documents cascades on network. It starts with any (un)directed, (un)weighted graph and a collection of documents and it outputs the propagation DAGs of the docs through the network. Diffusion process is guided by the nodes underlying preferences.

### TOC:
* [Demo](#Demo)
* [Help](#Help)
* [Usage](#Usage)
* [Analysis](#Analysis)
* [Statistics](#Statistics)

## Demo

The following code will run WoMG with the default parameters:

In [ ]:
%run ../src/womg_main.py

Checking the outputs

In [ ]:
!cd ../Output/ ; ls ; #cat Diffusion_formatted_output_sim0.txt



## Help

Let's check the parameters by the help page:

In [ ]:
!python3 ../src/womg_main.py --help

First set of quantitative parameters are:

1. number of topics to be considered in the topic distributions of documents and nodes interests; it has to be less than number of dimensions of the nodes' space provided by node2vec
2. number of documents TO BE GENERATED by lda, giving this parameter lda will be directly set to generative mode
3. steps of the diffusion simulation
4. H degree of homophily. Node2vec is used as baseline for generating interests vectors of the nodes starting from the given graph. Parameters *p* and *q* can achieve different decoded degree of homophily and structural equivalence (see paper). The best mix of them can be achieved only by a deep analysis of the network and a grid searh on the parameters. In order to pursuit generality in the input graph we use three degree of mixing: structural equivalence predominant, deepWalk (p=1, q=1), homophily predominant (which are not the best for representing the graph!).  1-H is the degree of social influence between nodes; which is the percentage of the avg interests vecs norms to be assigned to the influence vectors.
5. percentage of active nodes with respect to the total number of nodes in the intial configuration (before diffusion) for each doc.
6. virality of the doc; if virality is high, exponent of the power law is high and threshold for activation is low.


Next parameters concern input graph, input documents and the node2vec original parameters.

## Usage

The following code will produce a synthetic propagation dataset on the [Digg network dataset](https://www.isi.edu/~lerman/downloads/digg2009.html). This dataset consists in: graph dataset and diffusion dataset. We used the first as input of WoMG for generating diffusions and analyse results. 

We set:

1. the number of steps equal to 100
2. the maximum percentage of active nodes per doc equal to 0.065
3. number of generated docs equal to 3553
4. virality exponent of the docs distribution equal to 0.009

In [ ]:
#!python3 ../src/main.py --graph ../data/graph/digg/digg_edgelist.txt --directed --steps 100 --actives 0.065 --docs 3553  --virality 0.009

##### Output

The analysis of the actions using digg network as input can be done using simulation_index=_tutorial.

The real dataset analysis provides the following results:

    items actions [max, min, avg]:   6265 105 505

    users actions [max, min, avg]:   3415 20 115

## Analysis

In [ ]:
import ast
import pathlib
import numpy as np

In [ ]:
simulation_index = "_tutorial"
output_path = pathlib.Path.cwd() / "Output"

In [ ]:
file_info = output_path / str("Network_info_sim"+str(simulation_index)+".txt")
file_prop = output_path / str("Diffusion_formatted_output_sim"+str(simulation_index)+".txt")

###### import functions

In [ ]:
def extract(file_in):
    '''
    Returns file from the given input path
    '''
    if str(pathlib.Path(file_in).suffix) == '.txt':
        with open(file_in, 'r') as f:
            s = f.readlines()
    if str(pathlib.Path(file_in).suffix) == '.pickle':
        with open(file_in, 'rb') as f:
            s = pickle.load(f)
    return s

In [ ]:
def to_dict(inp, typ=False):
    '''
    if typ:
        Returns info input (inp) in dict format
    if not typ:
        Returns cascades in a dict format:
        (outer)first key: time, (inner)second key: item, 
        value: new active nodes
    '''
    
    if typ:
        info_dict = ast.literal_eval(str(inp).replace('[','').replace(']','').replace('"',''))
        out_dict = info_dict
        
    else:   
        prop_dict = {}
        index = 0
        for i in range(2, len(prop), 2):
            inp[i] = inp[i].replace('\n', '')
            inp[i] = inp[i].replace('set()','None')
            prop_dict[index] = ast.literal_eval(prop[i])
            index += 1

        out_dict = prop_dict
           
    return out_dict

In [ ]:
info_str = extract(file_info)
info = to_dict(info_str, typ=True)
prop = extract(file_prop)
cascades = to_dict(prop, typ=False)

##### analysis functions

In [ ]:
# items actions
def items_actions(cascades, plot=False):
    '''
    Returns the vector of all items' actions
    each entry is the the number of activations(actions)
    for the item identified by entry-index
    '''
    numb_docs = max(cascades[0].keys())
    items_action_vec = [0 for i in range(numb_docs+1)]
    for step in cascades.keys():
        for item in cascades[step].keys():
            if cascades[step][item] != None:
                items_action_vec[item] += len(cascades[step][item])
            #if cascades[step][item] == None:
                #print(item)
    #print(items_action_vec)
    if plot:
        plt.hist(items_action_vec)
        plt.show()
    
    print('items actions [max, min, avg]: ', int(max(items_action_vec)), int(min(items_action_vec)), int(np.mean(items_action_vec)))
    return items_action_vec

In [ ]:
items_data = items_actions(cascades)

In [ ]:
# users actions
def users_actions(cascades):
    '''
    Returns the vector of all users' actions
    each entry is the the number of activations(actions)
    for the user identified by entry-index
    '''
    numb_nodes = int(info['numb_nodes'])
 
        
    # defining dict
    users_actions_dict = {}
    for step in cascades.keys():
        for item in cascades[step].keys():
            if cascades[step][item] != None:
                for node in cascades[step][item]:
                    users_actions_dict[node] = 0
                    
    # counting            
    for step in cascades.keys():
        for item in cascades[step].keys():
            if cascades[step][item] != None:
                for node in cascades[step][item]:
                    users_actions_dict[node] += 1
                    
    users_actions_vec = [0 for i in range(numb_nodes)]
    for key, index in zip(sorted(users_actions_dict.keys()), range(numb_nodes)):
        users_actions_vec[index] = users_actions_dict[key]
    
    print('users actions [max, min, avg]: ',int(max(users_actions_vec)),
          int(min(users_actions_vec)), int(np.mean(users_actions_vec)))
    return users_actions_vec

In [ ]:
users_data = users_actions(cascades)

## Statistics

In [ ]:
import sys
import pathlib
path = pathlib.Path("../src")
sys.path.insert(0, str(path))
from womg_main import womg_main

help(womg_main)

In [ ]:
def simulate(numb_docs=None, viralities, actives, topics):
    '''
    Run womg with different sets of parameters

        Parameters
        ----------
        numb_docs : int
            number of documents to be generated
            (Default None: you need to pass a docs folder path)

        viralities : iterable/float
            array containing viralitiy params

        actives : iterable/float
            array containing actives_perc params
            (percentage of actives nodes for an items in the initial step)

        topics : iterable/int
            array containing numb_topics params
            
        
        Notes
        -----
        See womg reference for more details: help(womg_main)
        
    '''
    for v in viralities:
        for a in actives:
            for t in topics:      
                womg_main(numb_topics=t, actives_perc=a, virality=v, numb_docs=numb_docs)

In [ ]:
viralities = [2, 1, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05]
actives = [0.065]
topics = [15]

simulate(numb_docs=3553, 
         viralities=viralities, 
         actives=actives, 
         topics=topics
        )

In [ ]:
### virality plot

'''
file_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8]
viralities = [2, 1, 0.5, 0.3, 0.5, 0.35, 0.25, 0.2, 0.1]
'''
'''

file_idx = [0, 2, 6,  16]
viralities = [2, 0.5, 0.35,  0.11]
'''
path = '/Users/Cinus/University/Tesi/womg_git/Code/Library2.0/Output/Diffusion_formatted_output_sim'


file_idx = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
viralities = [0.5, 0.45, 0.4, 0.38, 0.35, 0.32, 0.3, 0.28, 0.25, 0.22, 0.2, 0.18, 0.15, 0.12]

y_item = []
x_item = []
y_user = []
x_user = []
cdf_data = {}
cdf_data['item'] = {}
cdf_data['user'] = {}
for i in tqdm(range(len(file_idx))):
    # data for cdf plot
    cdf_data['item'][i] = []
    cdf_data['user'][i] = []
    ########################
    file = path + str(file_idx[i]) + '.txt'
    prop = extract(file)
    cascades = to_dict(prop)
    items_data = items_actions(cascades, plot=False)
    users_data = users_actions(cascades)
    for n in range(len(items_data)):
        cdf_data['item'][i].append(items_data[n])
        y_item.append(items_data[n])
        x_item.append(viralities[i])
    for n in range(len(users_data)):
        cdf_data['user'][i].append(users_data[n])
        y_user.append(users_data[n])
        x_user.append(viralities[i])

##### violin plot

In [ ]:
fig = plt.figure(figsize=(15,11))

plt.xlabel('virality', fontsize=22)
plt.ylabel('items actions', fontsize=22)

ax = sns.violinplot(x=x_item, y=y_item)
sns.set()
plt.show()


fig.savefig('/Users/Cinus/University/Tesi/Tesi/Images/violin_pres.pdf')

##### cdf plot

In [ ]:
fig = plt.figure(figsize=(15,11))


file_idx = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
viralities = [0.5, 0.45, 0.4, 0.38, 0.35, 0.32, 0.3, 0.28, 0.25, 0.22, 0.2, 0.18, 0.15, 0.12]


cdf_hist = {}
cdf_hist['user'] = {}
cdf_hist['item'] = {}
for i in [2, 9, 13]:
    # evaluate the histogram
    cumulative = []
    values, base = np.histogram(cdf_data['item'][i], bins=50)
    cdf_hist['item'][i] = (values, base)
    
for i in [2, 9, 13]:
    #evaluate the cumulative
    cumulative = np.cumsum(cdf_hist['item'][i][0])/3553
    # plot the cumulative function
    plt.plot(base[:-1], cumulative, linewidth=5)
    #print(cumulative)
    plt.xlabel('actions', fontsize=33)
    plt.ylabel('CDF', fontsize=33)
    plt.tick_params(labelsize=17)
    #plt.yscale('log')
    #plt.xscale('log')
    plt.locator_params(axis='y', nbins=20)
    plt.locator_params(axis='x', nbins=20)
    plt.title('items CDF', fontsize=33)
    plt.legend(['virality-exp=0.5', 'virality-exp=0.3', 'virality-exp=0.11', 'virality-exp=0.45', 'virality-exp=0.45'], 
               fontsize=33)
plt.show()
#fig.savefig('/Users/Cinus/University/Tesi/Tesi/Images/cdf_items.pdf')
fig.savefig('/Users/Cinus/University/Tesi/Presentazioni/Tesi_Template/Downloads/cdf_items.pdf')